<H3>COLOUR SCHEME</H3>
<h5>Start Node: Orange <br>
    <br>
End Node: Turquoise<br>
    <br>
Path Node: Purple<br>
        <br>
Open Node: Light Green<br>
    <br>
Closed Node: Soft Red<br>
    <br>
Obstacle Node: Dark Grey<br>
    <br>
Grid Lines: Light Grey</h4>

<h3><b>Usage<b></b></h3>
    
<h4>1. Initialize Start and End Points:</h4><h5>Left-click on a cell to set it as the start (orange) node. Left-click again on a different cell to set it as the end (turquoise) node.</h5>

<h4>2. Toggle Obstacle Mode:</h4><h5>Press O on the keyboard to toggle Obstacle Mode on/off. When in obstacle mode, left-click on cells to turn them into obstacles (black), creating barriers.</h5>

<h4>3. Start Pathfinding:</h4><h5>Once you have defined both start and end points, press SPACE to begin the A* pathfinding algorithm. The algorithm will attempt to find the shortest path around obstacles from start to end.</h5>

<h4>4. Clear the Grid:</h4><h5>Press C to reset the grid to a blank state, allowing you to start a new pathfinding configuration.</h5>

In [ ]:
import pygame
import sys

pygame.init()
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("DFS Path Finding Visualization")

# Color scheme
SOFT_RED = (239, 83, 80)
SOFT_GREEN = (129, 199, 132)
SOFT_BLUE = (100, 181, 246)
SOFT_YELLOW = (255, 241, 118)
SOFT_WHITE = (250, 250, 250)
SOFT_BLACK = (55, 71, 79)
SOFT_PURPLE = (186, 104, 200)
SOFT_ORANGE = (255, 183, 77)
SOFT_GREY = (207, 216, 220)
SOFT_TURQUOISE = (128, 222, 234)

class Spot:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = SOFT_WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows

    def get_pos(self):
        return self.row, self.col

    def is_closed(self):
        return self.color == SOFT_RED

    def is_open(self):
        return self.color == SOFT_GREEN

    def is_barrier(self):
        return self.color == SOFT_BLACK

    def is_start(self):
        return self.color == SOFT_ORANGE

    def is_end(self):
        return self.color == SOFT_TURQUOISE

    def reset(self):
        self.color = SOFT_WHITE

    def make_start(self):
        self.color = SOFT_ORANGE

    def make_closed(self):
        self.color = SOFT_RED

    def make_open(self):
        self.color = SOFT_GREEN

    def make_barrier(self):
        self.color = SOFT_BLACK

    def make_end(self):
        self.color = SOFT_TURQUOISE

    def make_path(self):
        self.color = SOFT_PURPLE

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

    def update_neighbors(self, grid):
        self.neighbors = []
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier():  # Down
            self.neighbors.append(grid[self.row + 1][self.col])

        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier():  # Up
            self.neighbors.append(grid[self.row - 1][self.col])

        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier():  # Right
            self.neighbors.append(grid[self.row][self.col + 1])

        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier():  # Left
            self.neighbors.append(grid[self.row][self.col - 1])

    def __lt__(self, other):
        return False

def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()

def dfs(draw, grid, start, end):
    stack = [start]
    came_from = {}
    visited = {start}

    while stack:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        current = stack.pop()

        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True

        for neighbor in current.neighbors:
            if neighbor not in visited:
                visited.add(neighbor)
                came_from[neighbor] = current
                stack.append(neighbor)
                neighbor.make_open()

        draw()

        if current != start:
            current.make_closed()

    return False

def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Spot(i, j, gap, rows)
            grid[i].append(spot)
    return grid

def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, SOFT_GREY, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, SOFT_GREY, (j * gap, 0), (j * gap, width))

def draw(win, grid, rows, width):
    win.fill(SOFT_WHITE)
    for row in grid:
        for spot in row:
            spot.draw(win)
    draw_grid(win, rows, width)
    pygame.display.update()

def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos
    row = y // gap
    col = x // gap
    return row, col

def main(win, width):
    ROWS = 25
    grid = make_grid(ROWS, width)

    start = None
    end = None
    obstacle_mode = False

    run = True
    while run:
        draw(win, grid, ROWS, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if pygame.mouse.get_pressed()[0]:  # LEFT mouse button
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                if obstacle_mode:
                    if spot != start and spot != end:
                        spot.make_barrier()
                elif not start and spot != end:
                    start = spot
                    start.make_start()
                elif not end and spot != start:
                    end = spot
                    end.make_end()

            elif pygame.mouse.get_pressed()[2]:  # RIGHT mouse button
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                spot.reset()
                if spot == start:
                    start = None
                elif spot == end:
                    end = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start and end:
                    for row in grid:
                        for spot in row:
                            spot.update_neighbors(grid)
                    dfs(lambda: draw(win, grid, ROWS, width), grid, start, end)

                if event.key == pygame.K_c:
                    start = None
                    end = None
                    grid = make_grid(ROWS, width)

                if event.key == pygame.K_o:  # Toggle obstacle mode with "O"
                    obstacle_mode = not obstacle_mode
                    print(f"Obstacle Mode: {'ON' if obstacle_mode else 'OFF'}")

    pygame.quit()

main(WIN, WIDTH)
